<h1>MODEL TRAINER</h1>

In [32]:
#LOAD DEPENDENCIES
import os
import cv2
import time
import pickle
import logging
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from sklearn.utils import class_weight
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras import applications
from tensorflow.keras import Model, layers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.layers import AveragePooling2D, AlphaDropout, Activation, Add, BatchNormalization, Concatenate, Layer, ReLU, Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dense, Dropout

from tensorflow.keras.applications.efficientnet import EfficientNetB0 as trainable_model_a
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2 as trainable_model_b

#PREVENT ERROR UNCESSARY MESSAGES
tf.get_logger().setLevel(logging.ERROR)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [33]:
optimizerAdam = Adam(learning_rate= 0.01)
optimizerSGD = tf.keras.optimizers.SGD(learning_rate=0.00001, momentum=0.9, nesterov=True)  


optimizerLoad =[
    optimizerAdam,
    optimizerSGD,
]

#Set constants
batch_size = 16
epochs = 30
optimizer = optimizerLoad[1]
architecture = 'MFuReCNN_alpha_do_SGD_lr0.00001_aDo0.2_bz16_ep30'

DCNN_A = 'DCNN_A'
DCNN_B = 'DCNN_B'

In [ ]:
#LOAD THE DATA
# train_data_dir = "kvasir-dataset-v2/train"
validation_data_dir = "kvasir-dataset-v2/val"
test_data_dir = "kvasir-dataset-v2/test"
img_rows, img_cols = 224, 224
input_shape = (img_rows,img_cols,3)
model_input = Input(shape=input_shape)
print("Data folders found!")
print("The Input size is set to ", model_input) 

In [35]:
# Tiền xữ lý dữ liệu

# import cv2
# import os
# import shutil

# # Tạo thư mục con ( 8 lớp )
# def prepare_class_directories(input_dir, output_dir, class_names):
#     if not os.path.exists(output_dir):
#         os.makedirs(output_dir)
#     for class_name in class_names:
#         class_dir = os.path.join(output_dir, class_name)
#         if not os.path.exists(class_dir):
#             os.makedirs(class_dir)

# # Chức năng cắt ảnh và lưu vào đúng thư mục lớp
# def crop_images_in_directory(input_dir, output_dir, class_names, crop_x_left=30):
#     prepare_class_directories(input_dir, output_dir, class_names)

#     for class_name in class_names:
#         class_input_dir = os.path.join(input_dir, class_name)
#         class_output_dir = os.path.join(output_dir, class_name)

#         if os.path.exists(class_input_dir):
#             for file in os.listdir(class_input_dir):
#                 if file.endswith(('.png', '.jpg', '.jpeg')):
#                     file_path = os.path.join(class_input_dir, file)
#                     img = cv2.imread(file_path)
#                     if img is not None:
#                         h, w, _ = img.shape
#                         img_cropped = img[:, crop_x_left:w, :]  # Crop the image
#                         save_path = os.path.join(class_output_dir, file)
#                         cv2.imwrite(save_path, img_cropped)

# # Set up
# train_data_dir = "kvasir-dataset-v2/train"
# cropped_train_data_dir = "kvasir-dataset-v2/cropped_train"
# class_names = ['normal-cecum', 'normal-pylorus', 'normal-z-line', 'polyps', 
#                'ulcerative-colitis', 'esophagitis', 'dyed-lifted-polyps', 'dyed-resection-margins']

# # Thực hiện cắt ảnh trong thư mục train
# crop_images_in_directory(train_data_dir, cropped_train_data_dir, class_names, crop_x_left=250)


In [36]:
cropped_train_data_dir = "kvasir-dataset-v2/cropped_train"

In [ ]:
#DATA GENERATORS
#Augment Training Data
train_datagen = ImageDataGenerator(rescale = 1. / 255,
                                   horizontal_flip=True,
                                   vertical_flip=True,
                                   rotation_range=45,
                                   shear_range=0.1,
                                   zoom_range=0.1,
                                   height_shift_range=0.1,
                                   width_shift_range=0.1,
                                   fill_mode='constant',
                                   brightness_range=[0.1, 1.0])

# Tăng cường dữ liệu
# train_datagen = ImageDataGenerator(
#     rotation_range=10,
#     width_shift_range=0.1,
#     height_shift_range=0.1,
#     zoom_range=[0.8, 1.2],
#     horizontal_flip=True,
#     fill_mode='nearest'
# )

val_datagen = ImageDataGenerator(rescale = 1. / 255)

test_datagen = ImageDataGenerator(rescale = 1. / 255)

train_generator = train_datagen.flow_from_directory(
        cropped_train_data_dir,
        target_size=(img_rows,img_cols),
        batch_size=batch_size,
        class_mode='categorical',
        seed=42,
        classes=['normal-cecum', 
                 'normal-pylorus', 
                 'normal-z-line', 
                 'polyps', 
                 'ulcerative-colitis', 
                 'esophagitis', 
                 'dyed-lifted-polyps', 
                 'dyed-resection-margins'])

validation_generator = val_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_rows,img_cols),
        batch_size=batch_size,
        class_mode='categorical',
        seed=42,
        shuffle=False,
        classes=['normal-cecum', 
                 'normal-pylorus', 
                 'normal-z-line', 
                 'polyps', 
                 'ulcerative-colitis', 
                 'esophagitis', 
                 'dyed-lifted-polyps', 
                 'dyed-resection-margins'])

test_generator = test_datagen.flow_from_directory(
        test_data_dir,
        target_size=(img_rows,img_cols),
        batch_size=batch_size,
        class_mode='categorical',
        seed=42,
        shuffle=False,
        classes=['normal-cecum', 
                 'normal-pylorus', 
                 'normal-z-line', 
                 'polyps', 
                 'ulcerative-colitis', 
                 'esophagitis', 
                 'dyed-lifted-polyps', 
                 'dyed-resection-margins'])

#CHECK THE NUMBER OF SAMPLES
nb_train_samples = len(train_generator.filenames)
nb_validation_samples = len(validation_generator.filenames)
nb_test_samples = len(test_generator.filenames)


print("Train samples:", nb_train_samples)
print("Validation samples:", nb_validation_samples)
print("Test samples:", nb_test_samples)

if nb_train_samples == 0:
    print("NO DATA TRAIN FOUND! Please check your train data path and folders!")
else:
    print("Train samples found!")
    
if nb_validation_samples == 0:
    print("NO DATA VALIDATION FOUND! Please check your validation data path and folders!")
    print("Check the data folders first!")
else:
    print("Validation samples found!")
    
if nb_test_samples == 0:
    print("NO DATA TEST FOUND! Please check your test data path and folders!")
    print("Check the data folders first!")
else:
    print("Test samples found!")

#check the class indices
train_generator.class_indices
validation_generator.class_indices
test_generator.class_indices

#true labels
Y_test=validation_generator.classes
test_labels = test_generator.classes

num_classes= len(train_generator.class_indices)

print('Model set to train', num_classes, 'classes')

if nb_train_samples and nb_validation_samples and nb_test_samples > 0:
    print("Generators are set!")
    print("Check if dataset is complete and has no problems before proceeding.")

In [ ]:
# Model
#EfficientNetB0

builder_a = DCNN_A + '_builder'

#TRANSFER LEARNING
def builder_a(model_input):
    builder_a = trainable_model_a(weights='imagenet', 
                                    include_top=False, 
                                    input_tensor = model_input)

#PARTIAL LAYER FREEZING
    for layer in builder_a.layers:
        layer.trainable = False
        
    for layer in builder_a.layers:
        layer._name = layer.name + '_' + DCNN_A
        
    for BatchNormalization in builder_a.layers:
        BatchNormalization.trainable = False

    x = builder_a.layers[-17].output
    
#AUXILIARY FUSING LAYER (AuxFL)
    x = Conv2D(192, 1, padding='valid', activation='selu', kernel_initializer='lecun_normal')(x)
    x = AveragePooling2D(1, 1)(x)
    x = AlphaDropout(0.2)(x)

    dcnn_a = Model(inputs=builder_a.input, outputs=x, name=DCNN_A)
    return dcnn_a

#INITIALIZE THE MODEL
dcnn_a = builder_a(model_input)

#PLOT THE MODEL STRUCTURE
print("PLEASE CHECK THE ENTIRE MODEL UP TO THE END")

dcnn_a.summary()

print("successfully built!")

In [ ]:
# Model
#MobileNetV2

builder_b = DCNN_B + '_builder'

#TRANSFER LEARNING
def builder_b(model_input):
    builder_b = trainable_model_b(weights='imagenet', 
                                    include_top=False, 
                                    input_tensor = model_input)

#PARTIAL LAYER FREEZING
    for layer in builder_b.layers:
        layer.trainable = False
        
    for layer in builder_b.layers:
        layer._name = layer.name + '_' + DCNN_B
        
    for BatchNormalization in builder_b.layers:
        BatchNormalization.trainable = False
    
    x = builder_b.layers[-39].output
    
#AUXILIARY FUSING LAYER (AuxFL)
    x = Conv2D(192, 8, padding='valid', activation='selu', kernel_initializer='lecun_normal')(x)
    x = AveragePooling2D(1, 1)(x)
    x = AlphaDropout(0.2)(x)

    dcnn_b = Model(inputs=builder_b.input, outputs=x, name=DCNN_B)
    return dcnn_b

#INITIALIZE THE MODEL
dcnn_b = builder_b(model_input)

#PLOT THE MODEL STRUCTURE
print("PLEASE CHECK THE ENTIRE MODEL UP TO THE END")

dcnn_b.summary()

print(" successfully built!")

In [ ]:
#RE-INITIALIZE FOR FUSION
dcnn_a = builder_a(model_input)
dcnn_b = builder_b(model_input)

print("Accomplished Pre-training and ready for fusion")

In [ ]:
#FUSE THE MODELS INTO A SINGLE PIPELINE

models = [dcnn_a, 
          dcnn_b]

print("Fusion success!")
print("Ready to connect with its ending layers!")

In [ ]:
#Build the Fusion Residual Block (FuRB)
def mfurecnn_builder(models, model_input):
    outputs = [m.output for m in models]
    
#INITIAL FUSION LAYER
    y = Add(name='InitialFusionLayer')(outputs)
    
#FuRB
    y_bn1 = BatchNormalization()(y)
    y_selu1 = tf.keras.activations.selu(y_bn1)
    y_conv1 = Conv2D(192, 1, kernel_initializer='lecun_normal')(y_selu1)
    y_bn2 = BatchNormalization()(y_conv1)
    y_selu2 = tf.keras.activations.selu(y_bn2)
    y_conv2 = Conv2D(192, 1, kernel_initializer='lecun_normal')(y_selu2)
    
    y_merge = Add(name='FuRB')([y, y_conv2])
    
#FINE-TUNING
    y = GlobalAveragePooling2D()(y_merge)
    y = AlphaDropout(0.5)(y)
    prediction = Dense(num_classes,activation='softmax', name='Softmax_Classifier' + architecture)(y)
    model = Model(model_input, prediction, name=architecture)
    return model

#Istantitate the model and report the summary
mfurecnn = mfurecnn_builder(models, model_input)

print()
print()
print("PLEASE CHECK THE MODEL UP TO THE END")
print()
print()
print()

mfurecnn.summary()
print("Complete and ready for compilation and training!")

In [ ]:
last_conv_layer_name = mfurecnn.layers[-4].name

if last_conv_layer_name == 'ResidualFusionLayer':
    print("CORRECT LAYER SELECTED:", last_conv_layer_name)
else:
    print("INCORRECT LAYER SELECTED:", last_conv_layer_name)
    print("Please Reselect")

In [ ]:
import datetime
import pickle
import os
import time
import psutil
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np

# Generate timestamp
timestamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

#MODEL COMPILATION WITH HYPER-PARAMETERS, LOSS FUNCTIONS AND TRAINING!

mfurecnn.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy']) 

reduce_lr = ReduceLROnPlateau(monitor='val_acc', factor=0.5, patience=2,
                              verbose=1, mode='max')

callbacks = [reduce_lr]

# MODEL TRAINING
start_time = time.time()

history = mfurecnn.fit(train_generator, steps_per_epoch=nb_train_samples // batch_size,
                                  epochs=epochs, validation_data=validation_generator,
                                  callbacks=callbacks, 
                                  validation_steps=nb_validation_samples // batch_size, verbose=1)


elapsed_time = time.time() - start_time
train_time = time.strftime("%H:%M:%S", time.gmtime(elapsed_time))
memory_usage = psutil.Process(os.getpid()).memory_info().rss / (1024 ** 3)  # in GB
print()
print()
print(train_time, 'train_time')
print()
print(elapsed_time, 'Seconds')
print()
print()
print("MODEL SERIALIZING WAIT FOR A MOMENT...")
print()

#SAVE AFTER TRAINING COMPLETES
model_save_dir = f"models/{architecture}"
os.makedirs(model_save_dir, exist_ok=True)

model_filename = f"{model_save_dir}/{architecture}_{timestamp}_model.h5"
history_filename = f"{model_save_dir}/{architecture}_{timestamp}_history.pickle"
weights_filename = f"{model_save_dir}/{architecture}_{timestamp}.h5"
json_filename = f"{model_save_dir}/{architecture}_{timestamp}.json"
results_filename = f"{model_save_dir}/{architecture}_{timestamp}_results.txt"

mfurecnn.save(model_filename)

# Save the history for evaluation
def save_h(file, history):
    with open(file, 'wb') as file_pi:
        pickle.dump(history, file_pi)
    print(f"History saved to {file}")

save_h(history_filename, history.history)

print("\n\nThe Model weights and history are successfully trained and saved!\n")

In [ ]:
# Reload the trained model

model = load_model(model_filename)
print("The model is loaded")

In [ ]:
# Save model to JSON and weights
def save_m(weights_file, json_file, model):
    model.save_weights(weights_file)
    model_json = model.to_json()
    with open(json_file, "w") as json_file:
        json_file.write(model_json)
    print(f"Model weights saved to {weights_file}")
    print(f"Model structure saved to {json_file}")

save_m(weights_filename, json_filename, model)

In [ ]:
# Evaluate on test data
test_loss, test_acc = model.evaluate(test_generator, steps=nb_test_samples // batch_size)
Y_pred = model.predict(test_generator, steps=nb_test_samples // batch_size)
y_pred = np.argmax(Y_pred, axis=1)

# Confusion Matrix
cm = confusion_matrix(test_generator.classes, y_pred)
cr = classification_report(test_generator.classes, y_pred, target_names=test_generator.class_indices.keys())

# Save results
def save_results(filename, history, train_time, memory_usage, nb_train_samples, nb_validation_samples, nb_test_samples, test_acc, test_loss, cm, cr):
    with open(filename, 'w') as f:
        f.write(f"Training Time: {train_time}\n")
        f.write(f"Memory Usage: {memory_usage:.2f} GB\n")
        f.write(f"Number of Training Samples: {nb_train_samples}\n")
        f.write(f"Number of Validation Samples: {nb_validation_samples}\n")
        f.write(f"Number of Test Samples: {nb_test_samples}\n")
        f.write(f"Test Accuracy: {test_acc}\n")
        f.write(f"Test Loss: {test_loss}\n")
        
        f.write("\nTraining Accuracy and Loss per Epoch:\n")
        for epoch, (acc, loss) in enumerate(zip(history['accuracy'], history['loss'])):
            f.write(f"Epoch {epoch+1}: Accuracy={acc}, Loss={loss}\n")
        
        f.write("\nValidation Accuracy and Loss per Epoch:\n")
        for epoch, (val_acc, val_loss) in enumerate(zip(history['val_accuracy'], history['val_loss'])):
            f.write(f"Epoch {epoch+1}: Val Accuracy={val_acc}, Val Loss={val_loss}\n")
        
        f.write("\nConfusion Matrix:\n")
        f.write(np.array2string(cm))
        
        f.write("\n\nClassification Report:\n")
        f.write(cr)

    print(f"Results saved to {filename}")

save_results(results_filename, history.history, train_time, memory_usage, nb_train_samples, nb_validation_samples, nb_test_samples, test_acc, test_loss, cm, cr)

print("\nAll results have been saved!")

In [ ]:
# Save accuracy and loss per epoch
def save_accuracy_loss(filename, history):
    with open(filename, 'w') as f:
        f.write("Epoch\tAccuracy\tLoss\tVal Accuracy\tVal Loss\n")
        for epoch in range(len(history['accuracy'])):
            acc = history['accuracy'][epoch]
            loss = history['loss'][epoch]
            val_acc = history['val_accuracy'][epoch]
            val_loss = history['val_loss'][epoch]
            f.write(f"{epoch+1}\t{acc}\t{loss}\t{val_acc}\t{val_loss}\n")
    print(f"Accuracy and loss per epoch saved to {filename}")

# Add this to the end of your existing code after saving the model and history
accuracy_loss_filename = f"{model_save_dir}/{architecture}_{timestamp}_accuracy_loss.txt"
save_accuracy_loss(accuracy_loss_filename, history.history)
